In [61]:
import pandas as pd

### A partir d'un dataFrame cette methode va récuperer toute les cartes joués et en faire des phrases qui seront en suite passé à l'algorithme word2vec

### A games 
### On prend la colonne games

### A partir de card_history
### on recupere l'historique des cartes sorties pour chaque partie On filtre si le champs card_history existe ou si il est vide

### A partir de sentances
### Pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent", puis on recupère le nom des cartes pour crée des phrases.


In [ ]:
def createsentances(data):
    games = data["games"]
    cards = [e["card_history"] for e in games if e.has_key("card_history") and e["card_history"]!=[]]
    sentances = []
    for c in cards:
        sentances.append([e["card"]["name"]for e in c if e["player"] == "me"])
        sentances.append([e["card"]["name"]for e in c if e["player"] == "opponent"])
    return sentances

creation de phrase a partir de plusieurs fichier json
listfile est une liste de fichier avec leur chemin

In [ ]:
def createsentancesfiles(listfile):
    sentances = []
    for l in listfile:
        data=pd.read_json(l)
        sentances += createsentances(data)
        del data
    return sentances

In [ ]:
directoryHearthstoneGames = "HeathstoneGames/"
from os import listdir
from os.path import isfile, join
filesHearthstoneGames = [directoryHearthstoneGames+f for f in listdir(directoryHearthstoneGames) if isfile(join(directoryHearthstoneGames, f))]


In [ ]:
sentances = createsentancesfiles(filesHearthstoneGames)

In [62]:
data = pd.read_json("2018-01.zip");

In [ ]:
sentances = createsentances(data)


# Je prend l'url avec collectible car je ne veux que les cartes disponibles pour faire un deck

In [5]:
import requests

urlApi = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"

ApiHearthStone = requests.get(urlApi).json()

In [6]:
listNameCard = [e["name"] for e in ApiHearthStone]

on filtre les cartes des parties à seulement celle disponible pour la construction de deck

In [ ]:
sentances = [filter(lambda x: x in listNameCard, e) for e in sentances]

# On va utilisier gensim et plus particulièrement l'algo Word2vec

In [1]:
from gensim.models import Word2Vec

max_count exclue les mots dont leur nombre d'apparition est inférieure à celui indiqué
size nombre de noeuds
workers paralléliser sur un nombres de coeurs
ier nombre d'epochs

In [2]:
model = Word2Vec(sentances,min_count=10,size=200,workers=4,iter=100)

NameError: name 'sentances' is not defined

sauvegarde du model

In [ ]:
model.save("word2vecModel")

charger le modele entrainer et de continuer l'entrainement

In [3]:
model = Word2Vec.load("word2vecModel")

on filtre l'api car nous avons pas besoin des information sur les héros qui ne sont pas joué pas joués par les joueurs
on cree un dictionnaire pour faire correspondre le nom d'une carte à sa classe
utile apres pour verifier que les cartes suggere sont du meme classe que celle déjà dans le deck

In [63]:
cardKnowns = model.wv.vocab.keys()
ApiHearthStone = filter(lambda x:x["name"] in cardKnowns,ApiHearthStone)
cardClass = {x["name"]:x["cardClass"] for x in ApiHearthStone}

cardClass


{u'Lightspawn': u'PRIEST',
 u'Tortollan Primalist': u'NEUTRAL',
 u'Rattling Rascal': u'NEUTRAL',
 u'Scourgelord Garrosh': u'WARRIOR',
 u'Dragon Consort': u'PALADIN',
 u'Amani Berserker': u'NEUTRAL',
 u'Dragonling Mechanic': u'NEUTRAL',
 u'Temporus': u'PRIEST',
 u'Hand of Protection': u'PALADIN',
 u'Stormpike Commando': u'NEUTRAL',
 u"Anub'ar Ambusher": u'ROGUE',
 u'Shaku, the Collector': u'ROGUE',
 u'The Curator': u'NEUTRAL',
 u'Herald Volazj': u'PRIEST',
 u'Grimscale Chum': u'PALADIN',
 u'Scaled Nightmare': u'NEUTRAL',
 u'Volcanic Drake': u'NEUTRAL',
 u'Twig of the World Tree': u'DRUID',
 u'Primal Talismans': u'SHAMAN',
 u'Pit Snake': u'ROGUE',
 u'Flash Heal': u'PRIEST',
 u'Mirror Image': u'MAGE',
 u'Force of Nature': u'DRUID',
 u'Explosive Runes': u'MAGE',
 u'Biteweed': u'ROGUE',
 u'Journey Below': u'ROGUE',
 u'Holy Light': u'PALADIN',
 u'Water Elemental': u'MAGE',
 u'Drakonid Operative': u'PRIEST',
 u'Warsong Commander': u'WARRIOR',
 u'Windfury Harpy': u'NEUTRAL',
 u"King's Elekk": 

fonction qui creer un deck aleatoire
numberscards nombre de carte voulue
typecard le type de deck
cardClass dictionnaire de carte associer à leur classe

In [48]:
import random
def createDeck(cardClass,typecard,numbercards):
    deck = []
    n = [k for k,v in cardClass.iteritems() if(v==typecard or v == "NEUTRAL")]
    
    for i in range(numbercards):
        indice = random.randint(0,len(n)-1)
        deck.append(n[indice])
    return deck

proposition de cartes, si deck de taille 30 ou plus aucune proposition

In [41]:
def proposeCards(uncompletedDeck,model,cardsClass,nbcards=2):
    deckClass = ""
    uncompletedDeckInside =  list(uncompletedDeck)
    numberMaxCards = 30
    if len(uncompletedDeckInside) <1:
        raise ValueError("it need to have at least 1 card in the uncompleted deck")
    if nbcards < 1:
        raise ValueError("nbcards can't be under 1")
    if nbcards>10:
        nbcards = 10
    if len(uncompletedDeckInside) >= numberMaxCards:
        return
    
    #on verifie qu des le départ que 2 classes de cartes ne soient pas presente
    #exception à la classe NEUTRAL
    
    nbClasses = set([cardsClass[e] for e in myuncompletedDeck])
    if len(nbClasses)>2 or (len(nbClasses)==2 and "NEUTRAL" not in nbClasses):
       raise ValueError("Just one class can be present in the uncompleted deck.\n Exeception for neutrals cards")
    
    #on met a jour cardClass qui correpond à la classe du deck en fonction de la classe de cartes proposé 
    #si toutes les cartes sont neutre
    deckClass = [e for e in nbClasses if e != "NEUTRAL"]
    #si NEUTRAL
    if deckClass==[]:
        deckClass = ""
    #si classe
    else:
        deckClass = deckClass[0]
    
    #on va essayer de definir une classe de deck en fonction de la proposition on prend les premières jusqu'a une
    #carte non neutre
    
    negativeCards = []
    
    while len(uncompletedDeckInside)<numberMaxCards:
        proposes = model.wv.most_similar(positive=uncompletedDeckInside,negative=negativeCards,topn=5)
        proposes = [x[0] for x in proposes]
        if(deckClass==""):
            for e in proposes:
                if cardsClass[e] != "NEUTRAL":
                    deckClass = cardsClass[e]
                    break
        else:
            negativeCards += [e for e in proposes if cardsClass[e] != "NEUTRAL" and cardsClass[e]!=deckClass]
            proposes = [e for e in proposes if cardsClass[e] == "NEUTRAL" or cardsClass[e]==deckClass]
            
        diff = numberMaxCards-len(uncompletedDeckInside)
        if diff < nbcards :
            proposes = proposes[:diff]
       
        uncompletedDeckInside += proposes 
    return uncompletedDeckInside

In [46]:
myuncompletedDeck = ["The Lich King","Loatheb","Abomination"]

decks=[]
for i in range(1,3):
    decks.append(proposeCards(myuncompletedDeck,model,cardClass,nbcards=i))

In [185]:
myuncompletedDeck = createDeck(cardClass,"SHAMAN",10)
set([cardClass[e] for e in myuncompletedDeck])

{u'NEUTRAL', u'SHAMAN'}

In [47]:
for l in decks:
    print set([cardClass[e] for e in l])

set([u'NEUTRAL', u'PRIEST'])
set([u'NEUTRAL', u'PRIEST'])


In [74]:
myuncompletedDeck = createDeck(cardClass,"HUNTER",8)
decks=[]
for i in range(1,6):
    decks.append(proposeCards(myuncompletedDeck,model,cardClass,nbcards=i))

In [75]:
for l in decks:
    print set([cardClass[e] for e in l])

set([u'NEUTRAL', u'HUNTER'])
set([u'NEUTRAL', u'HUNTER'])
set([u'NEUTRAL', u'HUNTER'])
set([u'NEUTRAL', u'HUNTER'])
set([u'NEUTRAL', u'HUNTER'])
